In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import pywt
import sys
import torch

In [2]:
def bin_conv(df_id):
    scale = 16 ## equals to hexadecimal
    num_of_bits = 11
    bin_conv_out = []
    for i in np.array(df_id):
        bin_conv_out.append(bin(int(str(i), scale))[2:].zfill(num_of_bits))
    
    return bin_conv_out

In [3]:
def data_process(data, label):
    CreatData = []
    output = []
    data = pd.read_csv(data) # Load Data
    data = data[['CAN_ID', 'Time_Gap']] # Filter Data 
    data["Labele_num"] = np.repeat(label, (len(data))) # Label Data
    length = len(data)
    bin_out = bin_conv(data['CAN_ID']) # Binary Convert
    
    for number in bin_out: # Split binary into column
        output.append([bit for bit in number])
    can_id = pd.DataFrame(output, columns=['bin_' + str(i) for i in range(1, 12)])
    df = pd.concat([can_id, data[['Time_Gap']]], axis=1) # Filter X_train or X_test

    X = np.array(df) # Filter X_train or X_test for insert into tupple
    y = np.array(data[['Labele_num']]) # Filter y_train or y_test for insert into tupple
    strat_time = 0
    timestamp = 10

    for i in range(int(abs(length/10-1))):
        features = X[strat_time:timestamp]
        features = np.reshape(features, (1, 10, 12))
        CreatData.append((features, y[timestamp]))

        strat_time = strat_time + 10
        timestamp = timestamp + 10
        
    return CreatData

In [14]:
input_ = data_process("Kia_2_Normal.csv", 2) # Input Data File Name and Label number (Normal 0, DoS 1, Fuzz 2, Replay 3)
print("Total Data Length: ", len(input_))
print("Element Shape: ", input_[0][0].shape)

Total Data Length:  47320
Element Shape:  (1, 10, 12)


In [15]:
input_[0]

(array([[['0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0',
          0.23291015625],
         ['0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0',
          0.25390625],
         ['0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0',
          4.294189453125],
         ['0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0',
          0.23193359375],
         ['0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1',
          0.237060546875],
         ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
          0.240966796875],
         ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1',
          0.240966796875],
         ['0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0',
          0.235107421875],
         ['0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '1',
          0.23388671875],
         ['0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0',
          0.2509765625]]], dtype=object),
 array([2]))

In [6]:
train_dataset, test_dataset = train_test_split(input_, test_size=0.2)
print("Number of Training Sequences: ", len(train_dataset))
print("Number of Testing Sequences: ", len(test_dataset))

Number of Training Sequences:  37856
Number of Testing Sequences:  9464


In [7]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=False)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

# 